In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from scipy.interpolate import interp1d
import math
from features import implied_probability, american_to_decimal, fixed_length, split_for_datetime, time_for_75, fixed_length_timelin, linspace_stamps 

In [7]:
df = pd.read_csv("trainSequences.csv", header=None, names=range(231))
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)
df.head()

C:\Users\maxbr\AppData\Local\Temp\ipykernel_18544\637591706.py:1: DtypeWarning: Columns (192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("trainSequences.csv", header=None, names=range(231))


,fighter,event,date,opponent,timestep-1,timestep-2,timestep-3,timestep-4,timestep-5,timestep-6,...,timestep-218,timestep-219,timestep-220,timestep-221,timestep-222,timestep-223,timestep-224,timestep-225,timestep-226,timestep-227
0,Daichi Abe,UFC Fight Night 132: Cowboy vs. Edwards,2018-06-23,Li Jingliang,2018-06-13 12:09:00 - price: +240,2018-06-13 12:21:00 - price: +325,2018-06-13 13:42:00 - price: +315,2018-06-13 15:24:00 - price: +323,2018-06-13 16:18:00 - price: +320,2018-06-13 17:06:00 - price: +313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Juan Adams,UFC on ESPN 4: Dos Anjos vs. Edwards,2019-07-20,Greg Hardy,2019-06-30 17:33:00 - price: -155,2019-06-30 18:12:00 - price: +115,2019-06-30 19:48:00 - price: +110,2019-07-01 02:00:00 - price: +100,2019-07-01 02:24:00 - price: +120,2019-07-01 06:51:00 - price: +140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Klidson Abreu,UFC on ESPN+ 21: Zabit vs. Kattar,2019-11-09,Shamil Gamzatov,2019-10-28 12:36:00 - price: +190,2019-10-28 14:36:00 - price: +195,2019-10-28 22:06:00 - price: +168,2019-10-29 02:54:00 - price: +153,2019-10-29 05:21:00 - price: +136,2019-10-29 09:51:00 - price: +139,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ricardo Abreu,UFC Fight Night 68: Henderson vs. Boetsch,2015-06-06,Jake Collier,2015-05-30 19:18:00 - price: -210,2015-05-31 13:39:00 - price: -200,2015-05-31 16:24:00 - price: -195,2015-06-03 22:19:00 - price: -192,2015-06-04 10:00:00 - price: -187,2015-06-05 17:39:00 - price: -185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kevin Aguilar,UFC 262: Oliveira vs. Chandler,2021-05-15,Tucker Lutz,2021-04-26 17:20:00 - price: -110,2021-04-26 17:26:00 - price: -130,2021-04-26 19:46:00 - price: -125,2021-04-26 20:30:00 - price: -120,2021-04-26 21:32:00 - price: +100,2021-04-27 15:56:00 - price: +101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = df.dropna(thresh=4)
df = df.iloc[:, 4:]
df.head()

,timestep-1,timestep-2,timestep-3,timestep-4,timestep-5,timestep-6,timestep-7,timestep-8,timestep-9,timestep-10,...,timestep-218,timestep-219,timestep-220,timestep-221,timestep-222,timestep-223,timestep-224,timestep-225,timestep-226,timestep-227
0,2018-06-13 12:09:00 - price: +240,2018-06-13 12:21:00 - price: +325,2018-06-13 13:42:00 - price: +315,2018-06-13 15:24:00 - price: +323,2018-06-13 16:18:00 - price: +320,2018-06-13 17:06:00 - price: +313,2018-06-13 20:45:00 - price: +318,2018-06-13 23:54:00 - price: +315,2018-06-14 01:36:00 - price: +308,2018-06-14 09:18:00 - price: +305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-06-30 17:33:00 - price: -155,2019-06-30 18:12:00 - price: +115,2019-06-30 19:48:00 - price: +110,2019-07-01 02:00:00 - price: +100,2019-07-01 02:24:00 - price: +120,2019-07-01 06:51:00 - price: +140,2019-07-01 08:09:00 - price: +110,2019-07-01 10:30:00 - price: +115,2019-07-01 11:33:00 - price: +108,2019-07-01 12:51:00 - price: +103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-10-28 12:36:00 - price: +190,2019-10-28 14:36:00 - price: +195,2019-10-28 22:06:00 - price: +168,2019-10-29 02:54:00 - price: +153,2019-10-29 05:21:00 - price: +136,2019-10-29 09:51:00 - price: +139,2019-10-29 10:12:00 - price: +140,2019-10-29 12:06:00 - price: +153,2019-10-29 13:30:00 - price: +168,2019-10-29 17:15:00 - price: +167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-05-30 19:18:00 - price: -210,2015-05-31 13:39:00 - price: -200,2015-05-31 16:24:00 - price: -195,2015-06-03 22:19:00 - price: -192,2015-06-04 10:00:00 - price: -187,2015-06-05 17:39:00 - price: -185,2015-06-05 18:21:00 - price: -181,2015-06-06 09:53:00 - price: -179,2015-06-06 10:33:00 - price: -177,2015-06-06 17:18:00 - price: -175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-26 17:20:00 - price: -110,2021-04-26 17:26:00 - price: -130,2021-04-26 19:46:00 - price: -125,2021-04-26 20:30:00 - price: -120,2021-04-26 21:32:00 - price: +100,2021-04-27 15:56:00 - price: +101,2021-04-27 17:30:00 - price: +105,2021-04-27 19:50:00 - price: +103,2021-04-27 20:22:00 - price: -107,2021-04-27 21:26:00 - price: -106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
X = []
y = []
checklist = []

for index, row in df.iterrows():
    dfTest = df.loc[[index]]
    dfHold = dfTest

    dfTest = dfTest.dropna(axis=1)

    split_name = time_for_75(dfTest)
    split_index = dfTest.columns.get_loc(split_name)

    timestamps = linspace_stamps(dfTest)


    dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
            if isinstance(x, str) and 'price: ' in x else x
        )
    

    if split_index <8:
        continue
    
    target = float(dfTest.iloc[0, -1])
    sequence = dfTest.iloc[0, :split_index].values
    sequence = np.array(sequence, dtype=float)

    timestamps_seq = timestamps.iloc[0, :split_index].values
    timestamps_seq = np.array(timestamps_seq, dtype=float)
    
    last_value = sequence[-1]

    #0 means take the 48hr
    if target < last_value:
        target = 0
    #1 means take the closing 
    else:
        target = 1

    #decimal_seq = american_to_decimal(sequence)
    #impliedProb_seq = implied_probability(sequence)
    
    interpolated_seq = fixed_length_timelin(sequence, 100, 'nearest', timestamps_seq)
    interpolated_IP = implied_probability(interpolated_seq)

    #interpolated_dec = fixed_length(decimal_seq, 20, 'linear')
    #interpolated_imp = fixed_length(impliedProb_seq, 20, 'linear')

    #interpolated_all = np.concatenate((interpolated_seq, interpolated_dec, interpolated_imp))
    X.append(interpolated_IP)
    y.append(target)

C:\Users\maxbr\AppData\Local\Temp\ipykernel_18544\3125882617.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_18544\3125882617.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_18544\3125882617.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_18544\3125882617.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_18544\3125882617.py:17: FutureWarning: DataFrame.applymap has been deprecate

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train[0]))

100


In [26]:
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [27]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6049


In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.49      0.54       992
           1       0.61      0.71      0.65      1101

    accuracy                           0.60      2093
   macro avg       0.60      0.60      0.60      2093
weighted avg       0.60      0.60      0.60      2093



In [30]:
import joblib
joblib.dump(model, 'timeLinearNormalized100.pkl')

['timeLinearNormalized100.pkl']

In [11]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [12]:
model2 = Sequential([
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

In [17]:
model2.fit(X, y, epochs=200, batch_size=16, validation_split=0.2)

Epoch 1/200
404/404 [==============================] - 2s 6ms/step - loss: 0.6885 - accuracy: 0.5413 - val_loss: 0.6856 - val_accuracy: 0.5524
Epoch 2/200
404/404 [==============================] - 2s 5ms/step - loss: 0.6883 - accuracy: 0.5430 - val_loss: 0.6831 - val_accuracy: 0.5456
Epoch 3/200
404/404 [==============================] - 2s 5ms/step - loss: 0.6872 - accuracy: 0.5365 - val_loss: 0.6856 - val_accuracy: 0.5592
Epoch 4/200
404/404 [==============================] - 2s 5ms/step - loss: 0.6878 - accuracy: 0.5412 - val_loss: 0.6841 - val_accuracy: 0.5431
Epoch 5/200
404/404 [==============================] - 2s 5ms/step - loss: 0.6867 - accuracy: 0.5453 - val_loss: 0.6848 - val_accuracy: 0.5387
Epoch 6/200
404/404 [==============================] - 2s 5ms/step - loss: 0.6871 - accuracy: 0.5453 - val_loss: 0.6845 - val_accuracy: 0.5493
Epoch 7/200
404/404 [==============================] - 2s 5ms/step - loss: 0.6867 - accuracy: 0.5460 - val_loss: 0.6827 - val_accuracy: 0.5493

In [18]:
model2.save('nueralNetZero100.keras')